In [2]:
# import libary 

import pandas as pd 
import numpy as np

In [3]:
current = pd.read_csv('current.csv', index_col="sasdate")
# current.csv downloaded from 
# 'https://research.stlouisfed.org/econ/mccracken/fred-databases/'

month = pd.read_excel('PredictorData2021.xlsx', index_col = 'yyyymm')

markus = pd.read_csv('Macro.csv', index_col = 'Date')
# https://mpelger.people.stanford.edu/data-and-code

/Users/scarlett/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


## 1. "current.csv" : 125  macroeconimic variable transform 

In [5]:
current_tcode = list(current.values[0])[:]
current_ticker = current.columns.tolist()[:]
current1 = current.iloc[1:] # without transformation code

def transxf(x,tcode) :
    n=x.size
    small=1e-6
    if tcode==1: #  no transformation): x(t)
        y=x
        result=y
    elif tcode==2: # First difference: x(t)-x(t-1)
        y = x.diff()
        result= y
    elif tcode==3:  #Second difference: (x(t)-x(t-1))-(x(t-1)-x(t-2))
        y = x.diff()
        result= y ** 2
    elif tcode==4:    #Natural log: ln(x)
        if min(x) < small:
            y=np.nan
        else :
            y=np.log(x)
        result= y
    elif tcode==5:   #First difference of natural log: ln(x)-ln(x-1)
        y = np.log(x) - np.log(x.shift())
        result=y
    elif tcode==6:   #First difference of natural log: ln(x)-ln(x-1)
        #y = np.log(x) - np.log(x.shift())
        ln_diff1 = np.log(x) - np.log(x.shift())
        ln_diff2 = np.log(x.shift()) - np.log(x.shift(periods=2))
        result=ln_diff1 - ln_diff2
        #result=y**2 
    elif tcode==7 :  #First difference of percent change: (x(t)/x(t-1)-1)-(x(t-1)/x(t-2)-1)
        y=(x/x.shift(1) - 1) - (x.shift(1)/x.shift(2) - 1)
        result=y
    return result 

yt=[]                #Initialize output variable                   
N=current1.shape[1]  #Number of series kept
for i in range(0,N):   
    dum=transxf(current1.iloc[:,i],current_tcode[i])
    yt.append(dum)

trans_data=pd.DataFrame(yt).T
trans_data.columns=current1.columns
trans_data.index = current1.index
trans_data.to_csv('current_transformed.csv')  

In [7]:
trans_data_val = trans_data.copy()
trans_data_val

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
1/1/1959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2/1/1959,0.003931,0.003613,0.010368,0.007358,0.007310,0.019397,0.013406,0.008623,0.007309,0.005236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3/1/1959,0.006430,0.007359,0.009410,-0.003423,0.008321,0.014301,0.006033,0.004899,0.000000,0.019391,...,-0.001124,0.000262,-0.000022,-0.008147,0.004819,NaN,0.004929,0.004138,-0.014792,NaN
4/1/1959,0.006494,0.007049,-0.003639,0.019922,0.000616,0.021081,0.014337,0.014543,0.015655,0.006381,...,0.001349,0.001752,-0.000022,0.012203,-0.004890,NaN,0.012134,0.006734,0.024929,NaN
5/1/1959,0.005763,0.006616,0.012005,0.006797,0.007803,0.014951,0.008271,0.009583,0.004766,0.020154,...,-0.001687,-0.001937,-0.000021,-0.004090,-0.004819,NaN,0.002828,0.002020,-0.015342,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10/1/2022,0.004929,0.001923,0.003318,-0.001031,0.010616,0.000179,0.004620,0.007262,0.007908,0.018595,...,0.012495,-0.001477,0.000331,-0.001553,0.000375,1.3,-0.000678,0.003626,-0.005197,29.9215
11/1/2022,0.001898,0.001797,-0.003194,-0.009608,-0.010802,-0.005831,-0.005445,-0.006610,-0.005735,-0.028380,...,-0.008829,-0.000520,0.001026,-0.000624,0.003892,-3.1,0.002135,0.000955,0.009257,23.0367
12/1/2022,0.001156,0.001590,-0.003321,0.015158,-0.010998,-0.010236,-0.008693,-0.006204,-0.001970,-0.017382,...,-0.006173,0.001815,-0.001776,0.002399,-0.007070,2.9,-0.002817,-0.001225,0.007180,21.7702


## 2. "current_transformed.csv" Data Validation

In [9]:
trans_data_val = trans_data.copy()
trans_data_val = trans_data_val.loc['12/1/1966':'11/1/2016']
trans_data_val_col = trans_data_val.columns.tolist()

markus_col = markus.columns.tolist()

def common(a,b): 
    c = [value for value in a if value in b] 
    return c
common=common(trans_data_val_col,markus_col)

markus_new = markus[common]
trans_data_new = trans_data_val[common]

markus_new.set_index(trans_data_new.index, inplace = True)

for i in markus_new.columns.tolist():
    print(i,trans_data_new[i].corr(markus_new[i]))

RPI 0.9965293312516555
W875RX1 0.9954494450598225
DPCERA3M086SBEA 0.99739532880218
CMRMTSPLx 0.9992681466001035
RETAILx 0.9993448573686669
INDPRO 0.9959676452797654
IPFPNSS 0.9938216084796146
IPFINAL 0.9942182432874501
IPCONGD 0.9956825187356503
IPDCONGD 0.9980942389197349
IPNCONGD 0.991760450757054
IPBUSEQ 0.9943608537955572
IPMAT 0.9973695420437407
IPDMAT 0.997129728261442
IPNMAT 0.9970841875226232
IPMANSICS 0.9954701925285344
IPB51222S 0.9970484651869314
IPFUELS 0.9993822809941648
CUMFNS 0.9951804440510151
HWI 0.8385797445383888
HWIURATIO 0.9276791842764218
CLF16OV 0.9980404978595487
CE16OV 0.9995425397448461
UNRATE 0.9954467132414585
UEMPMEAN 0.9938633093685549
UEMPLT5 0.9988685120969618
UEMP5TO14 0.9994922917707812
UEMP15OV 0.9992001675975166
UEMP15T26 0.9977864179990886
UEMP27OV 0.9995380829879272
CLAIMSx 0.999667903458004
PAYEMS 0.9990535010682224
USGOOD 0.9997069354260556
CES1021000001 0.9994517578704234
USCONS 0.9996052123558187
MANEMP 0.9998789964703054
DMANEMP 0.999934395133

## 3. "PredictorData2021.xlsx" : 8 macroeconimic variable transform

In [10]:
variable = ['dp',
            'ep',
            'b/m',
            'ntis',
            'tbl',
            'tms',
            'dfy',
            'svar',
           ]
tcodes = [2,2,5,2,2,1,2,5]

month["dp"] = np.log(month["D12"]) - np.log(month["Index"])
month["ep"] = np.log(month["E12"]) - np.log(month["Index"])
month['tms'] = month['lty'] - month['tbl']
month['dfy'] = month['BAA'] - month['AAA']

month_new = month[variable]

def transxf(x,tcode) :
    n=x.size
    small=1e-6
    if tcode==1: #  no transformation): x(t)
        y=x
        result=y
    elif tcode==2: # First difference: x(t)-x(t-1)
        y = x.diff()
        result= y
    elif tcode==3:  #Second difference: (x(t)-x(t-1))-(x(t-1)-x(t-2))
        y = x.diff()
        result= y ** 2
    elif tcode==4:    #Natural log: ln(x)
        if min(x) < small:
            y=np.nan
        else :
            y=np.log(x)
        result= y
    elif tcode==5:   #First difference of natural log: ln(x)-ln(x-1)
        y = np.log(x) - np.log(x.shift())
        result=y
    elif tcode==6:   #First difference of natural log: ln(x)-ln(x-1)
        #y = np.log(x) - np.log(x.shift())
        ln_diff1 = np.log(x) - np.log(x.shift())
        ln_diff2 = np.log(x.shift()) - np.log(x.shift(periods=2))
        result=ln_diff1 - ln_diff2
        #result=y**2 
    elif tcode==7 :  #First difference of percent change: (x(t)/x(t-1)-1)-(x(t-1)/x(t-2)-1)
        y=(x/x.shift(1) - 1) - (x.shift(1)/x.shift(2) - 1)
        result=y
    return result

yt=[]                #Initialize output variable                   
N=month_new.shape[1]  #Number of series kept
for i in range(0,N):   
    dum=transxf(month_new.iloc[:,i],tcodes[i])
    yt.append(dum)
trans_macro = pd.DataFrame(yt).T

trans_macro.to_csv('macro8_transformed.csv')

/Users/scarlett/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## 4. "macro8_transformed.csv": Data Validation

In [11]:
trans_macro8 = trans_macro.copy()
trans_macro8 =trans_macro8.loc['196612': '201611']

Macro_markus = markus[variable]

Macro_markus.reset_index(inplace = True) 
trans_macro8.reset_index(inplace = True) 

for i in Macro_markus.columns.tolist()[1:]:
    print(Macro_markus[i].corr(trans_macro8[i]))

0.99954746936974
0.9997966571584396
0.9999999999999998
0.9997256637755887
0.9999984092084161
0.9999999261109774
1.0
0.9999999993189497


## 5. Merge Dataset: 
"current_transformed.csv"
"macro8_transformed.csv"

In [28]:
#current_trans = pd.read_csv('current_transformed.csv', index_col = 'sasdate')
#macro8_trans = pd.read_csv('macro8_transformed.csv', index_col ='yyyymm')
current_trans = trans_data
macro8_trans = trans_macro

current_trans = current_trans.loc['2/1/1980':'12/1/2021']
macro8_trans = macro8_trans.loc['198002':'202112']
macro8_trans.set_index(current_trans.index, inplace =True)
total_macro = pd.concat([current_trans, macro8_trans], axis=1)

In [29]:
total_macro.isnull().sum().sort_values(ascending = False)

ACOGNO       145
CP3Mx          2
COMPAPFFx      1
BAA            0
TB3SMFFM       0
            ... 
USTPU          0
SRVPRD         0
NDMANEMP       0
DMANEMP        0
svar           0
Length: 135, dtype: int64

In [30]:
total_macro.drop(columns = ['ACOGNO'], inplace = True)

missing_cols = ['CP3Mx','COMPAPFFx']
for i in missing_cols:
    total_macro[i].fillna(total_macro[i].mean, inplace = True)
    
total_macro.to_csv('total_macro_normalized.csv')

---

## 6. Get predicted return: get "firm_norm_with_predicted_return.pkl"

In [32]:
firm_norm = pd.read_pickle('8_2_chars_data_clean_rank_norm.pkl')
firm_norm.isnull().sum().sort_values(ascending = False)

ticker        627
jdate           0
nincr           0
pricedelay      0
pctacc          0
             ... 
chfeps          0
chempia         0
chcsho          0
chatoia         0
zerotrade       0
Length: 121, dtype: int64

In [33]:
firm_norm['predict_date'] = firm_norm['jdate']  + pd.offsets.MonthEnd(1)
firm_norm = firm_norm.sort_values(by=['permno','jdate'])

/var/folders/sg/y4ppv58972v4dp0byynbh0f00000gn/T/ipykernel_4245/4176852012.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  firm_norm['predict_date'] = firm_norm['jdate']  + pd.offsets.MonthEnd(1)


In [34]:
list_ = []
for i in range(firm_norm.shape[0]-1):
    now = firm_norm.iloc[i]
    later = firm_norm.iloc[i+1]
    if now['permno'] == later['permno'] and now['predict_date'] == later['jdate']:
        list_.append(later['ret_adj_ex'])
    else:
        list_.append(np.nan)
list_.append(np.nan)
firm_norm['predicted_return'] = list_

/var/folders/sg/y4ppv58972v4dp0byynbh0f00000gn/T/ipykernel_4245/2719080710.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  firm_norm['predicted_return'] = list_


In [35]:
# drop nan in predicted return 
firm_norm = firm_norm.dropna(subset=['predicted_return'])
firm_norm['predicted_return'].isnull().sum()

firm_norm = firm_norm.sort_index(ascending=True)
firm_norm.to_pickle('firm_norm_with_predicted_return.pkl')